# Introduction

This dataset includes several months (and counting) of data on daily trending YouTube videos. Data is is from videos in the United States, with up to 200 listed trending videos per day.

Each region’s data is in a separate file. Data includes the video title, channel title, publish time, tags, views, likes and dislikes, description, and comment count.

The associated JSON contains the categories for a specific video, which describes the category_id field.

---


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import json
%matplotlib inline

In [12]:
file = "C:/Users/Carter Carlson/Documents/Thinkful/Large Databases/Youtube/videos.csv"
#file = "C:/Users/ccarlsjh/Desktop/Important Files/Untitled Folder/Other/videos_small.csv"
df = pd.read_csv(file)

In [13]:
file = 'C:/Users/Carter Carlson/Documents/Thinkful/Large Databases/Youtube/category_id.json'
#file = "C:/Users/ccarlsjh/Desktop/Important Files/Untitled Folder/Other/category_id.json"
with open(file) as data_file:
    data = json.load(data_file)
num_list = []
title_list = []
for group in data['items']:
    num = group['id']
    title = group['snippet']['title']
    num_list.append(num)
    title_list.append(title)

In [14]:
cat_id_list = dict(zip(num_list, title_list))
df['category_title'] = [cat_id_list[str(i)] for i in df['category_id']]

In [15]:
# average views
avg_views = df.groupby('category_title')[['category_title','views']].mean()/1000000
avg_views

,views
category_title,
Autos & Vehicles,1.355965
Comedy,1.400372
Education,0.650565
Entertainment,1.874304
Film & Animation,2.487096
Gaming,2.012832
Howto & Style,0.880302
Music,5.131938
News & Politics,0.490310


In [16]:
# All category titles with mean views > 1 million
# NOTE - way to do this dynamically?  I want to create a df that only includes videos if their 
# category title has mean_views > 1 million

top_titles = []

In [17]:
# Music, Film & Animation, Science & Technology, Comedy
top_titles = ['Music', 'Film & Animation', 'Science & Technology', 'Comedy']
df1 = df.loc[df['category_title'].isin(top_titles)]

In [18]:
df1['category_title'][:10]

2                   Comedy
5     Science & Technology
7     Science & Technology
8         Film & Animation
12                   Music
15    Science & Technology
16                  Comedy
18    Science & Technology
21    Science & Technology
24                  Comedy
Name: category_title, dtype: object

In [20]:
views = df1.groupby('category_title')['views'].mean()
likes = df1.groupby('category_title')['likes'].mean()
likes/views * 1000 # likes per 1,000 views

category_title
Comedy                  43.128701
Film & Animation        23.519180
Music                   36.882082
Science & Technology    25.739061
dtype: float64

In [21]:
dislikes = df1.groupby('category_title')['dislikes'].mean()
dislikes/views * 1000 # dislikes per 1,000 views

category_title
Comedy                  1.422669
Film & Animation        0.925723
Music                   1.314244
Science & Technology    1.285546
dtype: float64

In [22]:
round((1-dislikes/likes) * 100,1) #average ratings 

category_title
Comedy                  96.7
Film & Animation        96.1
Music                   96.4
Science & Technology    95.0
dtype: float64

Hypothesis - disabling comments will change the overall likes and dislikes of a video.

- enabling/disabling comments probably doesn't have an impact on views.  People don't know if a video has comments enabled before viewing, so it shouldn't impact views in that sense.  It may impact views if people come back multiple times to comment on the same thread... sharing the video shouldn't change much whether there's comments or not

Test - 

Does disabling comments lead to disabling reviews or video removal?